# Linear Solver Performance: XDG Poisson, Single Core

### Part 1, Benchmark Setup and Execution

In [ ]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

In [ ]:
System.Security.Principal.WindowsIdentity.GetCurrent().Name

In [ ]:
#r "BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [ ]:
using BoSSS.Application.XdgPoisson3;

In [ ]:
string PROJECT_NAME = System.Environment.GetEnvironmentVariable("LinslvPerfSer") ?? "LinslvPerfSer"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

## Utility definitions

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }*/
}

## Init grids and save to database

In [ ]:
wmg.Grids

Create meshes in various resolutions:
- domain $\Omega = (-1,1)^3$; 
- a Dirichlet boundary is set everywhere; the challenge of this benchmark,
  from a numerical point, is the 1:1000 ratio in the diffusion coefficient

In [ ]:
int[] Resolutions_3D = new int[] { 2, 4, 8, 16, 32, 64 };
//int[] Resolutions_3D = new int[] { 2 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] zNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "-XdgPoisson_J" + J);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(GridName)); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            return "Dirichlet";
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

In [ ]:
grids

## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [ ]:
/*static class InitialValues {
    public static double RHS(double[] X) {
        return -Math.Sin(X[0]);
    }
    
    public static double DirichletBC(double[] X) {
        return 0.0;
    }
    
    public static double NeumannBC(double[] X) {
       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8)
           return 0;
       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8))
           return 0;

       return Math.Cos(10.0);
   }
}*/

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
// polynomial degrees to test
int[] PolyDegS = new int[] {2, 3, 5};

Solvers which we want to instrument:

In [ ]:
using BoSSS.Solution.AdvancedSolvers;

In [ ]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.direct_pardiso, 
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.pMultigrid
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [ ]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.direct_pardiso:
        case LinearSolverCode.direct_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 3000000; // Up to 3 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [ ]:
List<XdgPoisson3Control> controls = new List<XdgPoisson3Control>();
controls.Clear();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J*Np > GetMaxAllowedDOF(solver_name))
        continue;
    
    var ctrl = new XdgPoisson3Control();
    controls.Add(ctrl);

    string caseName = string.Format("XdgPoisson-J{0}_k{1}_{2}", J, k, solver_name);
    Console.WriteLine("setting up: " + caseName);
    ctrl.SessionName = caseName;
    
    ctrl.SetGrid(grd);
    ctrl.savetodb = true; //for debug's sake
    ctrl.SetDGdegree(k);
    
    ctrl.LinearSolver           = solver_name.GetConfig();
    var isc = ctrl.LinearSolver as IterativeSolverConfig;
    if(isc != null) {
        //Console.WriteLine(isc.ConvergenceCriterion);
        //ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
        isc.ConvergenceCriterion = 1e-8;
    }
    
    double radius           = 0.7;
    ctrl.ExcactSolSupported = false;
    ctrl.InitialValues.Add("Phi", new Formula("X => X[0].Pow2()+X[1].Pow2()+X[2].Pow2()-"+radius+".Pow2()"));
    ctrl.MU_A = -1;
    ctrl.MU_B = -1000;
    ctrl.InitialValues.Add("rhs#A", new Formula("X => 1"));
    ctrl.InitialValues.Add("rhs#B", new Formula("X => 1"));

    //ctrl.CutCellQuadratureType = XQuadFactoryHelper.MomentFittingVariants.Classic;
    //ctrl.SetDefaultDiriBndCnd  = true;
    //ctrl.ViscosityMode         = XLaplace_Interface.Mode.SIP;
        
    ctrl.AgglomerationThreshold = 0.1;
    
    ctrl.NoOfMultigridLevels = 100; // maximum number of multigrid levels to use; actual number will be far lower.
    ctrl.TracingNamespaces = "BoSSS.Solution";
}
}
}

In [ ]:
//string path = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XdgPoisson3\bin\Release\net5.0\bench";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, "control-" + ctrl.SessionName + ".obj"));
//}

Total number of simulations:

In [ ]:
controls.Count

## Launch Jobs

Use the default queue defined on this machine:

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

In [ ]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var j = ctrl.CreateJob();
    j.RetryCount = 1;
    j.Activate(myBatch);
    //ctrl.RunBatch();
}

In [ ]:
//foreach(var j in wmg.AllJobs.Values)
//    j.DeleteOldDeploymentsAndSessions();

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions.Where(sess => sess.Name.StartsWith("XdgPoisson_J"))

In [ ]:
var NoSuccess = controls.Select(ctrl => ctrl.GetJob()).Where(job => job.Status != JobStatus.FinishedSuccessful).ToArray();
NoSuccess

In [ ]:
// In the case of some failed job, print the directory name for further inspection:
foreach(var fail in NoSuccess)
   Console.WriteLine(fail + ":  @" + ((fail.LatestDeployment?.DeploymentDirectory?.FullName) ?? " no deployment directory"));
    //Console.WriteLine(fail.LatestDeployment);

In [ ]:
//foreach(var j in NoSuccess)
//    j.DeleteOldDeploymentsAndSessions();

In [ ]:
/*
string PathOffset = @"C:\Users\jenkinsci\Desktop\LinSlvPerfFail-20apr22";
foreach(var fail in NoSuccess) {
    var C = fail.GetControl();
    C.savetodb = false;
    C.SaveToFile(System.IO.Path.Combine(PathOffset, fail.Name + ".obj"));
    
    string Stdout = fail.Stdout;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stdout.txt"), Stdout);
    
    string Stderr = fail.Stderr;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stderr.txt"), Stderr);
}
*/

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("XdgPoisson") &&
                                        (Si.SuccessfulTermination == false
                                        || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions

#### Asserting Success:

Remark: since this is currently (22 Apr. 2022) work-in-progress, we allow for some jobs to fail; 
At this intermediate milestone, I want to record (by the means of tests) what **is already working**.
Thereby, I hope I won't break the working cases while trying to fix the failing ones.

In [ ]:
var prelim_allowedFails = new[] { "XdgPoisson-J*_k*_pMultigrid",
                                  "XdgPoisson-J4096_k2_exp_gmres_levelpmg",
                                  "XdgPoisson-J4096_k5_direct_pardiso",
                                  "XdgPoisson-J32768_k3_direct_pardiso"
                                };

In [ ]:
bool FailAllowed(string name) {
    foreach(string s in prelim_allowedFails) {
        if(s.WildcardMatch(name))
            return true;
    }
    return false;
}

In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Where(job => !FailAllowed(job.Name)).Count(), "Some Jobs Failed");

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Where(s => !FailAllowed(s.Name)).Count(), "Some Sessions did not terminate successfully.");

List Output of some job (arbitrarily the first one):

In [ ]:
wmg.AllJobs.First().Value.Stdout